In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pytorch_lightning as pl

# Download ``MNIST`` Dataset

In [2]:
# MNIST dataset and dataloaders
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root=os.getcwd(), train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)


100%|██████████| 9912422/9912422 [00:00<00:00, 12750438.32it/s]


Extracting /Users/adebayobraimah/Desktop/projects/fall2023project/MNIST/raw/train-images-idx3-ubyte.gz to /Users/adebayobraimah/Desktop/projects/fall2023project/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 26913062.39it/s]

Extracting /Users/adebayobraimah/Desktop/projects/fall2023project/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/adebayobraimah/Desktop/projects/fall2023project/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 7313845.02it/s]


Extracting /Users/adebayobraimah/Desktop/projects/fall2023project/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/adebayobraimah/Desktop/projects/fall2023project/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12443193.19it/s]

Extracting /Users/adebayobraimah/Desktop/projects/fall2023project/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/adebayobraimah/Desktop/projects/fall2023project/MNIST/raw



# Define Neural Network and ``LightningModule``

In [3]:
class NeuralNetwork(pl.LightningModule):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.model(x)
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)
    
    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = nn.CrossEntropyLoss()(outputs, targets)
        self.log('train_loss', loss)
        return loss

# Train the model

In [4]:
# Initialize the Lightning Trainer
model = NeuralNetwork()
trainer = pl.Trainer(accelerator='auto',max_epochs=5)  # Set max_epochs and gpus according to your environment

# Train the model using PyTorch Lightning
trainer.fit(model, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/adebayobraimah/Desktop/projects/fall2023project/lightning_logs

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 175 K 
-------------------------------------
175 K     Trainable params
0         Non-trainable params
175 K     Total params
0.701     Total estimated model params size (MB)
/Users/adebayobraimah/Desktop/projects/fall2023project/.env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
